<a href="https://colab.research.google.com/github/YoshiyukiKono/gen_ai-sandbox/blob/main/langchain_chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://book.st-hakky.com/docs/chains-of-langchain/

In [1]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


In [2]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


In [3]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.7 MB/s eta 0:00:00


In [5]:
import os
import openai
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass()
openai.api_key = os.getenv("OPENAI_API_KEY")

··········


In [6]:
# llm ラッパーのインポート
from langchain import OpenAI
# プロンプトテンプレートのインポート
from langchain.prompts import PromptTemplate

# LLMChain に加えて SimpleSequentialChain もインポートする
# SimpleSequentialChain は、複数のチェーンを連続実行するためのチェーンで以下の特徴をもつ
# - 各ステップの入出力は一つ
# - 各ステップの出力が次のステップの入力になる
from langchain.chains import LLMChain, SimpleSequentialChain

# llm ラッパーの初期化
llm = OpenAI(temperature=0)

# 最初のプロンプトテンプレートの作成
prompt_first = PromptTemplate(
    input_variables=["product"],
    template="{product}を作る会社の社名として、何かいいものはないですか？日本語の社名でお願いします。",
)

# 最初に実行する LLM チェーンを定義
# 会社名を考えてもらう
chain_first = LLMChain(llm=llm, prompt=prompt_first)

# 次のプロンプトテンプレートの作成
prompt_second = PromptTemplate(
    input_variables=["company_name"],
    template="{company_name}という会社名の企業のキャッチコピーを考えてください。",
)

In [7]:
# 次に実行する LLM チェーンを定義
# キャッチコピーを考えてもらう
chain_second = LLMChain(llm=llm, prompt=prompt_second)

# 二つの LLM チェーンを連結
overall_chain = SimpleSequentialChain(chains=[chain_first, chain_second], verbose=True)

# 連結してできたチェーンを実行
chatchphrase = prediction = overall_chain.run("カラフルな靴下")
print(chatchphrase)



> Entering new SimpleSequentialChain chain...


「カラフルソックス工房」


「足元を彩る、カラフルソックス工房」

> Finished chain.


「足元を彩る、カラフルソックス工房」


In [8]:
from typing import Dict, List

# LLMChain のインポート
from langchain.chains import LLMChain

# カスタムチェーンを定義するために Chain クラスをインポート
from langchain.chains.base import Chain


# このチェーンは、２つの LLMChain の出力を連結する
class ConcatenateChain(Chain):
    # もととなる２つの LLMChain をコンストラクタで受け取る
    chain_1: LLMChain
    chain_2: LLMChain

    @property
    def input_keys(self) -> List[str]:
        # ２つのチェーンの入力キーの和集合
        all_input_vars = set(self.chain_1.input_keys).union(set(self.chain_2.input_keys))
        return list(all_input_vars)

    @property
    def output_keys(self) -> List[str]:
        # このチェーンの出力キーは、concat_output のみ
        return ['concat_output']

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        # `output_keys` で定義したキーをもつ Dict を返す関数を定義する
        # ここでは、２つのチェーンを独立に実行した得られた出力を連結して返す
        output_1 = self.chain_1.run(inputs)
        output_2 = self.chain_2.run(inputs)
        return {'concat_output': output_1 + output_2}



In [9]:
# llm ラッパーのインポート
from langchain import OpenAI
# LLM チェーンの構築で必要なプロンプトテンプレートのインポート
from langchain.prompts import PromptTemplate

# llm ラッパーの初期化
llm = OpenAI(temperature=0)

prompt_1 = PromptTemplate(
    input_variables=["product"],
    template="{product}を作る会社の社名として、何かいいものはないですか？日本語の社名でお願いします。",
)
chain_1 = LLMChain(llm=llm, prompt=prompt_1)

prompt_2 = PromptTemplate(
    input_variables=["product"],
    template="{product}を作る会社のスローガンとして、何かいいものはないですか？日本語でお願いします。",
)
chain_2 = LLMChain(llm=llm, prompt=prompt_2)

concat_chain = ConcatenateChain(chain_1=chain_1, chain_2=chain_2)
concat_output = concat_chain.run("カラフルな靴下")
print(f"Concatenated output:\n{concat_output}")

Concatenated output:


「カラフルソックス工房」

「足元を彩る、カラフルな靴下」


https://python.langchain.com/docs/modules/chains/

## Async API
https://python.langchain.com/docs/modules/chains/how_to/async_chain

In [10]:
import asyncio
import time

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def generate_serially():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    for _ in range(5):
        resp = chain.run(product="toothpaste")
        print(resp)


async def async_generate(chain):
    resp = await chain.arun(product="toothpaste")
    print(resp)


async def generate_concurrently():
    llm = OpenAI(temperature=0.9)
    prompt = PromptTemplate(
        input_variables=["product"],
        template="What is a good name for a company that makes {product}?",
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    tasks = [async_generate(chain) for _ in range(5)]
    await asyncio.gather(*tasks)


s = time.perf_counter()
# If running this outside of Jupyter, use asyncio.run(generate_concurrently())
await generate_concurrently()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Concurrent executed in {elapsed:0.2f} seconds." + "\033[0m")

s = time.perf_counter()
generate_serially()
elapsed = time.perf_counter() - s
print("\033[1m" + f"Serial executed in {elapsed:0.2f} seconds." + "\033[0m")



Brightshine Toothpaste.


SmileFresh.


PearlySmiles Toothpaste Co.


BrightSmile Toothpaste Company


BrightSmile Toothpaste Co.
Concurrent executed in 0.76 seconds.


SparkleSmile Toothpaste.


Brush Bright Toothpaste.


SmileGlow Toothpaste


MintySmile Toothpaste Co.


Bright Smile Fresheners
Serial executed in 3.36 seconds.
